In [3]:
import numpy as np
import pandas as pd

In [4]:
bill_summaries = pd.read_csv('../data/bill_summaries.csv')
bioinfo = pd.read_csv('../data/bioinfo.csv')
fec_ids = pd.read_csv('../data/fec_ids.csv')
ideology = pd.read_csv('../data/ideology.csv')
sponsored_legislation = pd.read_csv('../data/sponsored_legislation.csv')
terms = pd.read_csv('../data/terms.csv')
vote_compare = pd.read_csv('../data/vote_compare.csv')
contrib = pd.read_csv('../data/contrib.csv')

1. Do each of the CSVs have a primary key? If so, what?

In [5]:
bill_summaries.columns


Index(['actionDate', 'actionDesc', 'currentChamber', 'currentChamberCode',
       'lastSummaryUpdateDate', 'text', 'updateDate', 'versionCode',
       'bill.congress', 'bill.number', 'bill.originChamber',
       'bill.originChamberCode', 'bill.title', 'bill.type',
       'bill.updateDateIncludingText', 'bill.url'],
      dtype='object')

In [6]:
bill_summaries[['bill.type', 'bill.number', 'versionCode']].duplicated().value_counts()

False    2751
Name: count, dtype: int64

In [7]:
bioinfo.columns

Index(['bioguide_id', 'Full name', 'Chamber', 'State', 'Party', 'District',
       'birthYear', 'image', 'Office address', 'Phone', 'Website'],
      dtype='object')

In [8]:
bioinfo[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [9]:
fec_ids.columns

Index(['bioguide_id', 'fec_id'], dtype='object')

In [10]:
fec_ids[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [11]:
ideology.columns

Index(['bioname', 'chamber', 'left_right_ideology', 'party_code',
       'state_abbrev', 'district_code', 'icpsr', 'bioguide_id', 'party'],
      dtype='object')

In [12]:
ideology[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [13]:
sponsored_legislation.columns

Index(['introducedDate', 'type', 'url', 'number', 'title', 'bioguide_id'], dtype='object')

In [14]:
sponsored_legislation[['url']].duplicated().value_counts()

False    14379
Name: count, dtype: int64

In [15]:
terms.columns

Index(['bioguide_id', 'chamber', 'congress', 'stateCode', 'startYear',
       'endYear', 'district'],
      dtype='object')

In [16]:
terms[['bioguide_id', 'chamber', 'congress']].duplicated().value_counts()

False    3257
Name: count, dtype: int64

In [17]:
vote_compare.columns

Index(['bioname', 'comparison_member', 'agree'], dtype='object')

In [18]:
vote_compare[['bioname', 'comparison_member']].duplicated().value_counts()

False    206040
Name: count, dtype: int64

In [19]:
contrib.columns

Index(['contributor_name', 'contributor_aggregate_ytd', 'memo_text', 'pdf_url',
       'fec_committee_id', 'fec_id'],
      dtype='object')

In [20]:
contrib[['pdf_url']].duplicated().value_counts()

True     431028
False    217665
Name: count, dtype: int64

In [21]:
contrib.loc[contrib.duplicated('pdf_url', keep=False)].sort_values('pdf_url').head(10)['pdf_url'][600174]

'http://docquery.fec.gov/cgi-bin/fecimg/?10930084691'

In [22]:
contrib = contrib.drop_duplicates()

In [23]:
contrib[['pdf_url']].duplicated().value_counts()

True     378715
False    217665
Name: count, dtype: int64

2. Do any of the tables need to become multiple tables, or combined into one table? 

Should be combined into one table (share the same primary key):bioinfo, fec_ids, ideology.

In [24]:
members = pd.merge(bioinfo, fec_ids,
                   on = 'bioguide_id',
                   how = 'outer',
                   validate = 'one_to_one',
                   indicator = 'matched')

In [25]:

members['matched'].value_counts()

matched
both          545
left_only       0
right_only      0
Name: count, dtype: int64

In [26]:
members = members.drop('matched', axis=1)


In [27]:
members = pd.merge(members, ideology,
                   on = 'bioguide_id',
                   how = 'outer',
                   validate = 'one_to_one',
                   indicator = 'matched')

In [28]:
members['matched'].value_counts()

matched
both          545
left_only       0
right_only      0
Name: count, dtype: int64

In [29]:
members = members.drop('matched', axis=1)

In [30]:
members.columns


Index(['bioguide_id', 'Full name', 'Chamber', 'State', 'Party', 'District',
       'birthYear', 'image', 'Office address', 'Phone', 'Website', 'fec_id',
       'bioname', 'chamber', 'left_right_ideology', 'party_code',
       'state_abbrev', 'district_code', 'icpsr', 'party'],
      dtype='object')

In [31]:
members.head(3).T

,0,1,2
bioguide_id,A000055,A000148,A000369
Full name,Robert B. Aderholt,Jake Auchincloss,Mark E. Amodei
Chamber,House of Representatives,House of Representatives,House of Representatives
State,Alabama,Massachusetts,Nevada
Party,Republican,Democratic,Republican
District,4.0,4.0,2.0
birthYear,1965.0,1988.0,1958.0
image,https://www.congress.gov/img/member/a000055_20...,https://www.congress.gov/img/member/67817e391f...,https://www.congress.gov/img/member/a000369_20...
Office address,"272 Cannon House Office Building, Washington, ...","1524 Longworth House Office Building, Washingt...","104 Cannon House Office Building, Washington, ..."
Phone,(202) 225-4876,(202) 225-5931,(202) 225-6155


In [32]:
members = members.drop(['Chamber', 'Party', 'District', 'State'], axis=1)

In [33]:
members.columns = [c.lower().replace(' ', '_') for c in members.columns]

In [34]:
members.head(3).T

,0,1,2
bioguide_id,A000055,A000148,A000369
full_name,Robert B. Aderholt,Jake Auchincloss,Mark E. Amodei
birthyear,1965.0,1988.0,1958.0
image,https://www.congress.gov/img/member/a000055_20...,https://www.congress.gov/img/member/67817e391f...,https://www.congress.gov/img/member/a000369_20...
office_address,"272 Cannon House Office Building, Washington, ...","1524 Longworth House Office Building, Washingt...","104 Cannon House Office Building, Washington, ..."
phone,(202) 225-4876,(202) 225-5931,(202) 225-6155
website,https://aderholt.house.gov/,https://auchincloss.house.gov,https://amodei.house.gov
fec_id,H6AL04098,H0MA04192,H2NV02395
bioname,"ADERHOLT, Robert","AUCHINCLOSS, Jake","AMODEI, Mark E."
chamber,House,House,House


In [35]:
members.to_csv('../data/thirdNF/members.csv', index=False)

In [36]:
bill_summaries.head(3).T

,0,1,2
actionDate,2025-10-15,2025-10-08,2025-10-08
actionDesc,Introduced in Senate,Introduced in House,Introduced in House
currentChamber,Senate,House,House
currentChamberCode,S,H,H
lastSummaryUpdateDate,2025-10-20T19:23:02Z,2025-10-20T14:56:29Z,2025-10-20T14:06:32Z
text,<p><strong>Shutdown Fairness Act</strong></p><...,<p><strong>Federal Worker Childcare Protection...,<p>This bill requires the federal government t...
updateDate,2025-10-20T19:23:15Z,2025-10-20T14:56:58Z,2025-10-20T14:07:00Z
versionCode,0,0,0
bill.congress,119,119,119
bill.number,3012,5720,5705


In [37]:
for c in bill_summaries.columns:
        print(c)
        d = bill_summaries.groupby(['bill.type', 'bill.number']).agg({c: 'nunique'})
        print(np.mean(d[c]) == 1)

actionDate
False
actionDesc
False
currentChamber
False
currentChamberCode
False
lastSummaryUpdateDate
False
text
False
updateDate
False
versionCode
False
bill.congress
True
bill.number
True
bill.originChamber
True
bill.originChamberCode
True
bill.title
True
bill.type
True
bill.updateDateIncludingText
True
bill.url
True


In [38]:
bills = bill_summaries[['bill.type', 'bill.number', 'bill.congress', 'bill.originChamber', 
                       'bill.originChamberCode', 'bill.title', 'bill.updateDateIncludingText',
                       'bill.url']].drop_duplicates()
bills.columns = [c.lower().replace('.', '_') for c in bills.columns]
bills.to_csv('../data/thirdNF/bills.csv')

In [39]:
bill_versions = bill_summaries.drop(['bill.congress', 'bill.originChamber', 
                       'bill.originChamberCode', 'bill.title', 'bill.updateDateIncludingText',
                       'bill.url'], axis=1)

In [40]:
bill_versions .columns = [c.lower().replace('.', '_') for c in bill_versions.columns]
bill_versions.to_csv('../data/thirdNF/bill_versions.csv')

In [41]:
vote_compare.head(3).T

,0,1,2
bioname,"GRASSLEY, Charles Ernest","GRASSLEY, Charles Ernest","GRASSLEY, Charles Ernest"
comparison_member,"MARKEY, Edward John","SCHUMER, Charles Ellis (Chuck)","WYDEN, Ronald Lee"
agree,0.021053,0.07193,0.052632


In [42]:
vote_compare.to_csv('../data/thirdNF/vote_compare.csv', index=False)

In [45]:
sponsored_legislation.head(15)

,introducedDate,type,url,number,title,bioguide_id
0,2025-09-10,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575
1,2025-09-09,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575
2,2025-09-09,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575
3,2025-09-09,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575
4,2025-09-09,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575
5,2025-09-09,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575
6,2025-06-25,HR,https://api.congress.gov/v3/bill/119/hr/4147?f...,4147.0,Poarch Band of Creek Indians Parity Act,R000575
7,2025-06-09,HR,https://api.congress.gov/v3/bill/119/hr/3838?f...,3838.0,Streamlining Procurement for Effective Executi...,R000575
8,2025-04-08,HR,https://api.congress.gov/v3/bill/119/hr/2740?f...,2740.0,To modify the boundaries of the Talladega Nati...,R000575
9,2025-03-31,HR,https://api.congress.gov/v3/bill/119/hr/2519?f...,2519.0,To provide a per diem allowance for Members of...,R000575


In [47]:
bills = pd.read_csv('../data/thirdNF/bills.csv')
bills.head(3)
#sponsored_legislation(['url'])

,Unnamed: 0,bill_type,bill_number,bill_congress,bill_originchamber,bill_originchambercode,bill_title,bill_updatedateincludingtext,bill_url
0,0,S,3012,119,Senate,S,Shutdown Fairness Act,2025-10-21,https://api.congress.gov/v3/bill/119/s/3012?fo...
1,1,HR,5720,119,House,H,Federal Worker Childcare Protection Act of 2025,2025-10-20,https://api.congress.gov/v3/bill/119/hr/5720?f...
2,2,HR,5705,119,House,H,To authorize the reimbursement by the Federal ...,2025-10-20,https://api.congress.gov/v3/bill/119/hr/5705?f...


In [ ]:
#build a checker to see if truly 1 to 1 
bills2 = pd.merge(bills, sponsored_legislation,
                  left_on='bill_url',
                  right_on='url',
                  how = 'outer',
                  validate = 'one_to_one',
                  indicator= 'matched')
bills2['matched'].value_counts()

matched
right_only    11644
both           2735
left_only         0
Name: count, dtype: int64

In [55]:
bills2.query("matched == 'right_only'")


,Unnamed: 0,bill_type,bill_number,bill_congress,bill_originchamber,bill_originchambercode,bill_title,bill_updatedateincludingtext,bill_url,introducedDate,type,url,number,title,bioguide_id,matched
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-10,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000612,right_only
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-10,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,R000575,right_only
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-25,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,O000175,right_only
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-25,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,O000175,right_only
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-25,NaN,https://api.congress.gov/v3/amendment/119/hamd...,NaN,NaN,S000185,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-24,SRES,https://api.congress.gov/v3/bill/119/sres/91?f...,91.0,A resolution acknowledging the third anniversa...,S001181,right_only
14371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-25,SRES,https://api.congress.gov/v3/bill/119/sres/93?f...,93.0,A resolution expressing the sense of the Senat...,D000563,right_only
14372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-25,SRES,https://api.congress.gov/v3/bill/119/sres/94?f...,94.0,An original resolution authorizing expenditure...,M000355,right_only
14373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-25,SRES,https://api.congress.gov/v3/bill/119/sres/95?f...,95.0,A resolution expressing support for the design...,S001217,right_only


In [60]:
pd.Series([['amendment' in x for x in bills2['url']]).value_counts()

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (406679638.py, line 1)

In [62]:
bills2 = bills2[['bill_type', 'bill_number', 'bill_congress', 'bill_originchamber',  'bill_originchambercode','bill_title', 'bill_updatedateincludingtext', 'introducedDate', 'bioguide_id']]

In [ ]:
bills2.shape

(14379, 9)

In [64]:
bills2.to_csv('../data/thirdNF/bills.csv', index=False)

In [65]:
terms.head(30)

,bioguide_id,chamber,congress,stateCode,startYear,endYear,district
0,R000575,House of Representatives,108,AL,2003,2005,3.0
1,R000575,House of Representatives,109,AL,2005,2007,3.0
2,R000575,House of Representatives,110,AL,2007,2009,3.0
3,R000575,House of Representatives,111,AL,2009,2011,3.0
4,R000575,House of Representatives,112,AL,2011,2013,3.0
5,R000575,House of Representatives,113,AL,2013,2015,3.0
6,R000575,House of Representatives,114,AL,2015,2017,3.0
7,R000575,House of Representatives,115,AL,2017,2019,3.0
8,R000575,House of Representatives,116,AL,2019,2021,3.0
9,R000575,House of Representatives,117,AL,2021,2023,3.0


In [67]:
terms.groupby('congress').agg({'startYear': 'std', 'endYear': 'std'})
# pd.crosstab(terms['congress'], terms['startYear'])

,startYear,endYear
congress,,
94,NaN,NaN
95,0.000000,0.000000
96,0.000000,0.000000
97,0.000000,0.000000
98,0.000000,0.000000
99,0.000000,0.000000
100,0.000000,0.000000
101,0.000000,0.000000
102,0.000000,0.000000


In [69]:
terms.query('congress == 111')['endYear'].value_counts()

endYear
2011    123
2009      1
Name: count, dtype: int64

In [70]:
contrib.head(3)

,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url,fec_committee_id,fec_id
0,FARMER'S & MERCHANTS BANK,920.34,NaN,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,NaN,H2AL03032
1,MCLAUGHLIN AND ASSOCIATES,40869.75,NaN,http://docquery.fec.gov/cgi-bin/fecimg/?259710...,NaN,H2AL03032
2,"DRUCKER LAWHON, LLP",38500.00,NaN,https://docquery.fec.gov/cgi-bin/fecimg/?20221...,NaN,H2AL03032


In [75]:
contrib.duplicated().value_counts()

False    596380
Name: count, dtype: int64

In [76]:
contrib.to_csv('../data/thirdNF/contrib.csv', index=False)

In [77]:
def pandas_df_to_dbml(df: pd.DataFrame, table_name: str) -> str:
    """
    Converts a pandas DataFrame to a DBML string.

    Args:
        df: The pandas DataFrame to convert.
        table_name: The name of the table in the DBML schema.

    Returns:
        A DBML string representing the DataFrame schema.
    """

    dbml_string = f"Table {table_name} {{\n"

    for column_name, column_type in df.dtypes.items():
        dbml_type = map_pandas_dtype_to_dbml_type(column_type)
        dbml_string += f"  {column_name} {dbml_type}\n"

    dbml_string += "}\n"
    return dbml_string

In [78]:
def map_pandas_dtype_to_dbml_type(dtype) -> str:
    """Maps a pandas dtype to a DBML type."""
    dtype_name = str(dtype)
    if "int" in dtype_name:
      return "int"
    if "float" in dtype_name:
      return "float"
    if "datetime" in dtype_name:
        return "datetime"
    return "varchar"

In [79]:
bill_versions = pd. read_csv('../data/thirdNF/bill_versions.csv')

In [80]:
print(pandas_df_to_dbml(bill_versions, "bill_versions"))

Table bill_versions {
  Unnamed: 0 int
  actiondate varchar
  actiondesc varchar
  currentchamber varchar
  currentchambercode varchar
  lastsummaryupdatedate varchar
  text varchar
  updatedate varchar
  versioncode int
  bill_number int
  bill_type varchar
}



In [81]:
bills = pd. read_csv('../data/thirdNF/bills.csv')

In [82]:
print(pandas_df_to_dbml(bills, "bills"))

Table bills {
  bill_type varchar
  bill_number float
  bill_congress float
  bill_originchamber varchar
  bill_originchambercode varchar
  bill_title varchar
  bill_updatedateincludingtext varchar
  introducedDate varchar
  bioguide_id varchar
}



In [83]:
members = pd. read_csv('../data/thirdNF/members.csv')

print(pandas_df_to_dbml(members, "members"))

Table members {
  bioguide_id varchar
  full_name varchar
  birthyear float
  image varchar
  office_address varchar
  phone varchar
  website varchar
  fec_id varchar
  bioname varchar
  chamber varchar
  left_right_ideology float
  party_code int
  state_abbrev varchar
  district_code int
  icpsr int
  party varchar
}



In [ ]:
#Need to go back and do the terms table
terms = pd.read_csv('../data/thirdNF/terms.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../data/thirdNF/terms.csv'

In [88]:
vote_compare = pd. read_csv('../data/thirdNF/vote_compare.csv')
print(pandas_df_to_dbml(vote_compare, "vote_compare"))

Table vote_compare {
  bioname varchar
  comparison_member varchar
  agree float
}

